In [1]:
import numpy as np
import pandas as pd
import sqlite3 as sq
import re
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [2]:
def label_cleaner(text):
        
    text = re.sub( r'ЖАЛОБА: на ', '', text) # замена слов 'ЖАЛОБА: на '
    text = re.sub( r'ЖАЛОБА: ', '', text) # замена слов 'ЖАЛОБА: на '
    text = re.sub( r'Жалоба на ', '', text) # замена слов 'ЖАЛОБА: на '
    text = re.sub( r'Жалобы на ', '', text) # замена слов 'ЖАЛОБА: на '
    
    #stw = ['ЖАЛОБА: ', 'ЖАЛОБА: на', 'Жалоба на', 'Жалобы на']
    #remove = r'('+'|'.join(stw)+')'
    ##remove = r'\b('+'|'.join(stw)+')\\b'
    #text = re.sub(remove, '', text)    

    text = str.strip(text.lower())
    return  text

def text_cleaner(text):
    text = text.lower() # приведение в lowercase,
    
    text = re.sub( r'https?://[\S]+', ' url ', text) # замена интернет ссылок
    text = re.sub( r'[\w\./]+\.[a-z]+', ' url ', text) 
 
    # text = re.sub( r'\d+[-/\.]\d+[-/\.]\d+', ' date ', text) # замена даты и времени
    # text = re.sub( r'\d+ ?гг?', ' date ', text) 
    # text = re.sub( r'\d+:\d+(:\d+)?', ' time ', text) 

    # text = re.sub( r'@\w+', ' tname ', text ) # замена имён twiter
    # text = re.sub( r'#\w+', ' htag ', text ) # замена хештегов

    text = re.sub( r'<[^>]*>', ' ', text) # удаление html тагов
    text = re.sub( r'[\W]+', ' ', text ) # удаление лишних символов

    stemmer = SnowballStemmer("russian")
    singles = [stemmer.stem(word) for word in text.split()]
    text = ' '.join(singles)

    # stw = ['в', 'по', 'на', 'из', 'и', 'или', 'не', 'но', 'за', 'над', 'под', 'то',
    #        'a', 'at', 'on', 'of', 'and', 'or', 'in', 'for', 'at' ]
    # remove = r'\b('+'|'.join(stw)+')\b'
    # text = re.sub(remove,' ', text)
    
    # text = re.sub( r'\b\w\b', ' ', text ) # удаление отдельно стоящих букв

    text = re.sub( r'\b\d+\b', ' digit ', text ) # замена цифр 

    return  text

def train_test_split( data, validation_split = 0.2):
    dict_data = data.to_dict('list')
    sz = len(dict_data['TOPIC'])
    indices = np.arange(sz)
    np.random.shuffle(indices)

    X = [ dict_data['DESCRIPTION'][i] for i in indices ]
    Y = [ dict_data['TOPIC'][i] for i in indices ]
    nb_validation_samples = int( validation_split * sz )

    return { 
        'train': { 'x': X[:-nb_validation_samples], 'y': Y[:-nb_validation_samples]  },
        'test': { 'x': X[-nb_validation_samples:], 'y': Y[-nb_validation_samples:]  }
    }

In [3]:
#Loading data
raw_data = pd.read_csv('complaints.csv', header = 0, sep = ';')
#raw_data.info()
#raw_data.head()
print(raw_data.shape)

(22840, 21)


In [4]:
#Preprocessing
raw_data = raw_data.dropna(subset=['TOPIC']) 
valuable_columns = ["TOPIC", "DESCRIPTION"]
raw_data["TOPIC"] = raw_data["TOPIC"] + " - " + raw_data["UNDER_TOPIC"].fillna('NA_U_TOPIC')

raw_data = raw_data[valuable_columns]

print(raw_data.shape)

raw_data = raw_data[(raw_data.TOPIC.str.contains("ЖАЛОБА: ") |
                     raw_data.TOPIC.str.contains("Жалоба на ") |
                     raw_data.TOPIC.str.contains("Жалобы на "))] #, na=False

print(raw_data.shape)

raw_data["TOPIC"] = [ label_cleaner(t) for t in raw_data["TOPIC"]]

print("\tколичество исходных категорий:", len(set(raw_data["TOPIC"])))
print(set(raw_data["TOPIC"]))

(22805, 2)
(19415, 2)
	количество исходных категорий: 101
{'обслуживание - кредиты', 'обслуживание - депозиты', 'обслуживание - терминалы', 'обслуживание - не рассказали', 'услугу "хранитель" - не рассказали', 'обслуживание - карта не закрылась', 'го/филиал/отделения/микроофисы/тт - некомфортное помещение', 'корреспонденцию банка - cмс по предложениям xsell', 'услугу "хранитель" - не возвращена сумма продукта', 'обслуживание - комиссии/тарифы банка', 'услугу "защита семьи"  - навязали', 'го/филиал/отделения/микроофисы/тт - неудобное месторасположение', 'корреспонденцию банка - прочее', 'услугу "защита семьи"  - некорректно проинформировали', 'услугу "хранитель" - фронт', 'обслуживание - не возвращена сумма продукта', 'корреспонденцию банка - кредиты', 'карточные продукты - na_u_topic', 'обслуживание - экстра лимит', 'услугу "хранитель" - na_u_topic', 'го/филиал/отделения/микроофисы/тт - корреспонденция (письма)', 'не согласие с условиями договора, %%, задолженностью, штрафами, тарифами

In [5]:
#Splitting to train and test
D = train_test_split(raw_data, 0.2)

print("\tразмер тестовой выборки:", len(D['test'] ['y']))
print("\tразмер тренировочной выборки:", len(D['train'] ['y']))

	размер тестовой выборки: 3883
	размер тренировочной выборки: 15532


In [6]:
#Learnming a classificator
print("[i] обучение классификатора...")

    # text_clf = Pipeline([
    #                ('hashvect', HashingVectorizer() ),
    #                ('tfidf', TfidfTransformer(use_idf=False )),
    #                ('clf', SGDClassifier(loss='hinge')),
    #                ])
    #
    # text_clf = Pipeline([
    #                ('covect', CountVectorizer() ),
    #                ('tfidf', TfidfTransformer(preprocessor=text_cleaner,use_idf=False )),
    #                ('clf', SGDClassifier(loss='hinge')),
    #                ])


text_clf = Pipeline([
                ('tfidf', TfidfVectorizer()),
                ('clf', SGDClassifier(loss='hinge')),
                ])

text_clf.fit(D['train']['x'], D['train']['y'])

print("[i] обучение завершено!")

[i] обучение классификатора...


C:\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


[i] обучение завершено!


In [7]:
#Testing and checking results
print("[i] тестируем...")

predicted = text_clf.predict( D['train']['x'] )
print("\taccuracy train: ", accuracy_score(  D['train']['y'] , predicted) )
    
predicted = text_clf.predict( D['test']['x'] )
print("\taccuracy test: ", accuracy_score(  D['test']['y'] , predicted) )

[i] тестируем...
	accuracy train:  0.8315735256245171
	accuracy test:  0.5207313932526397
